In [3]:
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm


torch.manual_seed(1234)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
def load_data_100k(path='./', delimiter='\t'):
    
    # Get the name of movies for inference.
    movies = np.genfromtxt(path + 'u.item', dtype='str', usecols=1, delimiter='|', encoding='ISO-8859-1')
    
    # Train and test data (10 movies for each user in test) contains: 
    # user id, movie id, rating and timestamp
    x_train = np.loadtxt(path + 'ua.base', skiprows=0, delimiter=delimiter).astype('int32')
    y_train = np.loadtxt(path + 'ua.test', skiprows=0, delimiter=delimiter).astype('int32')
    
    x_val = np.loadtxt(path + 'ub.base', skiprows=0, delimiter=delimiter).astype('int32')
    y_val = np.loadtxt(path + 'ub.test', skiprows=0, delimiter=delimiter).astype('int32')

    total = np.concatenate((x_train, y_train, x_val, y_val), axis=0)

    users_num = np.unique(total[:,0]).size
    movies_num = np.unique(total[:,1]).size
    
    x_train_ratings_num = x_train.shape[0]
    y_train_movies_num = y_train.shape[0]
    
    x_val_ratings_num = x_val.shape[0]
    y_val_movies_num = y_val.shape[0]
    
    # Observe that for train data we have movies' ratings for every user.
    # For test data, we have every users' rated movies.
    x_train_ratings = np.zeros((movies_num, users_num), dtype='float32')
    y_train_movies = np.zeros((users_num, movies_num), dtype='float32')
    
    x_val_ratings = np.zeros((movies_num, users_num), dtype='float32')
    y_val_movies = np.zeros((users_num, movies_num), dtype='float32')
        
    # For x_train and x_val:
    
    # In single matrix, maps each user's movie rating.
    # i.e. train[mov_id, user_id] = rating
    # Timestamp is discarded.
    for i in range(x_train_ratings_num):
        x_train_ratings[x_train[i, 1] - 1, x_train[i, 0] - 1] = x_train[i, 2]
        
    for i in range(x_val_ratings_num):
        x_val_ratings[x_val[i, 1] - 1, x_val[i, 0] - 1] = x_val[i, 2]

    # For y_train and y_val data: 
    
    # Store every user's rated movie. 
    # If it has rating > 3 put 1, else 0.
    # Discard movie ratings and timestamps.
    for i in range(y_train_movies_num):
        y_train_movies[y_train[i, 0] - 1, y_train[i, 1] - 1] = 1 if y_train[i, 2] > 3 else 0
    
    for i in range(y_val_movies_num):
        y_val_movies[y_val[i, 0] - 1, y_val[i, 1] - 1] = 1 if y_val[i, 2] > 3 else 0
    
    print(f'num of users: {users_num}')
    print(f'num of movies: {movies_num}')
    print(f'num of x_train ratings: {x_train_ratings_num}')
    print(f'num of y_train ratings: {y_train_movies_num}')

    return movies, movies_num, users_num, x_train_ratings, y_train_movies, x_val_ratings, y_val_movies

In [5]:
# Data Load

DATA_PATH = '../data/raw/ml_100k/'

try:
    MOVIES_NAMES, MOVIES_NUM, USERS_NUM, X_TRAIN_RATINGS, Y_TRAIN_MOVIES, \
        X_VAL_RATINGS, Y_VAL_MOVIES = load_data_100k(path=DATA_PATH, delimiter='\t')
except Exception as e:
    print('Error: Unable to load data', e)

num of users: 943
num of movies: 1682
num of x_train ratings: 90570
num of y_train ratings: 9430


In [6]:
print(X_TRAIN_RATINGS)
print(Y_TRAIN_MOVIES)

[[5. 4. 0. ... 5. 0. 0.]
 [3. 0. 0. ... 0. 0. 5.]
 [4. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Architecture of the model

In [7]:
LR = 0.01
EPOCHS = 1000
STOPPING_PATIENCE = 5 # number of consecutive rounds of early stopping condition before actual stop 
STOPPING_TOLERANCE = 1e-4 # minimum threshold for the difference between consecutive values of validation loss, to increase number of rounds
K_MOVIES = 10 # how many movies to recommend.

In [8]:
class MoviesRecommender(torch.nn.Module):
    """
    Model which outputs predictions for every movie that the user might like, 
    based on the given ratings of other movies by this user.
    """

    def __init__(self, movies_num):
        """
        k_movies: how many movies to recommend.
        """
        
        super(MoviesRecommender, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(movies_num, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, movies_num),
            nn.Softmax(dim=0)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
recommender_model = MoviesRecommender(MOVIES_NUM).to(device)

In [8]:
optimizer = torch.optim.Adam(recommender_model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

In [9]:
last_cross_entropy_loss = np.inf
early_stopping_counter = 0 
for i in tqdm(range(EPOCHS)):
    
    # Train model    
    optimizer.zero_grad()
    
    loss = 0.
    for usr_id in range(USERS_NUM):
                        
        # Send train ratings and masks of user to GPU
        ratings = torch.Tensor(X_TRAIN_RATINGS[:, usr_id]).to(device)
        y_true = torch.Tensor(Y_TRAIN_MOVIES[usr_id, :]).to(device)

        # Turn on the gradients calculation (train mode)
        recommender_model.train()

        # Get predicted movies and calculate the loss
        pred = recommender_model(ratings)

        loss += loss_fn(pred, y_true)

    loss.backward()
    optimizer.step()
    
    # Validation
    
    recommender_model.eval()
    
    epoch_loss = 0.
    for usr_id in range(USERS_NUM):
        pred = recommender_model(torch.Tensor(X_VAL_RATINGS[:, usr_id]).to(device))
        y_true = torch.Tensor(Y_VAL_MOVIES[usr_id, :]).to(device)

        val_loss = loss_fn(pred, y_true)

        # Update counter for early stopping
        early_stopping_counter = early_stopping_counter + 1 \
            if last_cross_entropy_loss - val_loss < STOPPING_TOLERANCE else 0

        last_cross_entropy_loss = val_loss
        
        epoch_loss += val_loss
    epoch_loss /= USERS_NUM

    print(f'Epoch: {i + 1} | Val loss: {epoch_loss}')
    
    # Check if we have waited enough iterations for early stopping.
    if STOPPING_PATIENCE == early_stopping_counter:
        print("EARLY STOPPING...")
        break

  0%|          | 1/1000 [00:03<50:37,  3.04s/it]

Epoch: 1 | Val loss: 42.763729095458984


  0%|          | 2/1000 [00:04<33:55,  2.04s/it]

Epoch: 2 | Val loss: 42.73106002807617


  0%|          | 3/1000 [00:05<28:04,  1.69s/it]

...

Epoch: 998 | Val loss: 42.70424270629883


100%|█████████▉| 999/1000 [22:00<00:01,  1.31s/it]

Epoch: 999 | Val loss: 42.70427322387695


100%|██████████| 1000/1000 [22:01<00:00,  1.32s/it]

Epoch: 1000 | Val loss: 42.70399856567383


# Save model

In [9]:
MODEL_PATH = "../models/"
# torch.save(recommender_model.state_dict(), MODEL_PATH + 'my_movie_recommender.pt')

# Inference.

In [10]:
# Load model
recommender_model = MoviesRecommender(MOVIES_NUM).to(device)
recommender_model.load_state_dict(torch.load(MODEL_PATH + 'my_movie_recommender.pt', map_location=device))
recommender_model.eval()

MoviesRecommender(
  (layers): Sequential(
    (0): Linear(in_features=1682, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1682, bias=True)
    (5): Softmax(dim=0)
  )
)

In [12]:
user_ratings = torch.zeros(MOVIES_NUM).to(device)

print("Input two integeres: index of movie and it's rating (1 to 5) separated by a single space.")
print("To stop writing, press ENTER.")

watched_movies = set()
while True:
    user_input = input()
    
    if user_input == '':
        break
    
    movie_index, rating = list(map(int, user_input.split()))
    watched_movies.add(movie_index)
    user_ratings[movie_index] = rating

pred = recommender_model(user_ratings)

# Output top K movies.
for i in range(K_MOVIES):
    recommended_movie = pred.argmax()
    
    if recommended_movie not in watched_movies:
        print(f'{i + 1}. {MOVIES_NAMES[recommended_movie]}')
        i += 1
    
    pred[recommended_movie] = 0

Input two integeres: index of movie and it's rating (1 to 5) separated by a single space.
To stop writing, press ENTER.
1. L.A. Confidential (1997)
2. Star Wars (1977)
3. Liar Liar (1997)
4. Titanic (1997)
5. Schindler's List (1993)
6. Trainspotting (1996)
7. Saint, The (1997)
8. Jerry Maguire (1996)
9. Princess Bride, The (1987)
10. English Patient, The (1996)
